# King County Housing Price Prediction From Linear Regression

## Overview
This notebook is an implementation of the linear regression model created from `housing_price_prediction.ipynb`. This notebook will import the model and a test data set with unknown prices. It will perform all feature engineering performed in `housing_price_prediction.ipynb` and output a list of predicted price. This predicted price will be combined to the test data and exported as seprate file in the results folder. 

### Libraries Import

In [1]:
import os
import pandas as pd
import numpy as np
import pickle 
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

### Test Data Import

In [2]:
kc_import_df = pd.read_csv("data/kc_house_data_test_features.csv", index_col=0)
kc_test_df = kc_import_df #this is done not to adulterate the original file
kc_test_df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


### Zipcode Dummy Series Import
The zipcode dummy variables must be imported from the original notebook to match the dimesionality of linear models to the dataset

In [3]:
ziplist = pd.read_csv("data/zipcod_dummy.csv", index_col=0)
ziplist = ziplist.zipcode

In [4]:
ziplist.head()

0    98058
1    98115
2    98006
3    98034
4    98052
Name: zipcode, dtype: int64

## Feature Engineering
This section is equivalent to feature engineering in `housing_price_prediction.ipynb`

In [5]:
#renovation
kc_test_df["renovated"] = kc_test_df.yr_renovated.apply(lambda x: 1 if x > 0 else 0)
kc_test_df["renovation_age"] = kc_test_df.yr_renovated.apply(lambda x: 2020-x if x > 0 else 0)

#basement
kc_test_df["basement"] = kc_test_df.sqft_basement.apply(lambda x: 1 if x != 0 else 0)

#master bathroom
kc_test_df["master_bathroom"] = kc_test_df.bathrooms.apply(lambda x: 1 if x > 2 else 0)

#family house
kc_test_df["family_house"] = kc_test_df.bedrooms.apply(lambda x: 1 if x > 2 else 0)

#sold year and quarter
kc_test_df["sale_year"] = kc_test_df.date.apply(lambda x: int(x[:4]))
kc_test_df["sale_quarter"] = kc_test_df.date.apply(lambda x: int(x[4:6])//3.1 + 1)

#zipcode dummy variables
kc_test_df = kc_test_df.merge(pd.get_dummies(ziplist), left_index=True, right_index=True)

#squared bedrooms and bathrooms
kc_test_df["bedroom_squared"] = kc_test_df["bedrooms"] ** 2
kc_test_df["bathroom_squared"] = kc_test_df["bathrooms"] ** 2

# uncomment to check the data set
# kc_test_df.head()

features = [col for col in kc_test_df.columns if col not in ["id", "date"] ] #remove unused column
kc_test_df_features = kc_test_df[features] #set train/test data using feature above

In [6]:
kc_test_df_features.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,renovated,renovation_age,basement,master_bathroom,family_house,sale_year,sale_quarter,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,bedroom_squared,bathroom_squared
0,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,0,1,1,1,2014,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,6.2500
1,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,0,1,1,1,2015,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,6.2500
2,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,0,0,1,1,1,2014,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,6.2500
3,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,0,0,1,0,1,2014,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,3.0625
4,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,0,0,0,1,1,2015,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,7.5625


In [7]:
kc_test_df_features.describe()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,renovated,renovation_age,basement,master_bathroom,family_house,sale_year,sale_quarter,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,bedroom_squared,bathroom_squared
count,4323.000000,4323.000000,4323.000000,4.323000e+03,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000
mean,3.387231,2.322056,2183.344899,1.275018e+04,1.738723,0.006477,0.201249,3.248439,7.904696,1936.424474,246.920426,1985.545686,64.198705,98077.777932,47.560232,-122.209611,2031.962757,10789.035855,0.032154,0.751561,0.366181,0.651399,0.861901,2014.320148,2.530187,0.016655,0.009022,0.014111,0.016655,0.008328,0.023363,0.008328,0.011566,0.004164,0.008328,0.007402,0.008790,0.011103,0.024520,0.003701,0.021050,0.009484,0.013185,0.011335,0.013648,0.005552,0.018274,0.030072,0.027065,0.002545,0.015036,0.023595,0.011566,0.026602,0.017812,0.011797,0.018968,0.022207,0.019431,0.014342,0.005783,0.013417,0.021513,0.014573,0.008096,0.017349,0.003932,0.026139,0.009947,0.015267,0.010409,0.007171,0.004164,0.010872,0.027990,0.012491,0.022669,0.021513,0.008559,0.015267,0.017349,0.017118,0.024057,0.013879,0.017349,0.011797,0.003238,0.023132,0.011335,0.013879,0.012954,0.011566,0.006708,0.013185,0.015730,12.294009,6.003426
std,0.906016,0.782063,966.768796,4.063943e+04,0.586193,0.080228,0.711334,0.554978,1.167461,895.247498,408.555950,30.572267,352.271856,50.974730,0.133668,0.147649,727.968964,27415.348930,0.176428,4.943671,0.481815,0.476582,0.345043,0.466587,1.039347,0.127990,0.094563,0.117961,0.127990,0.090885,0.151072,0.090885,0.106934,0.064400,0.090885,0.085727,0.093354,0.104798,0.154675,0.060731,0.143568,0.096935,0.114081,0.105872,0.116038,0.074311,0.133957,0.170804,0.162290,0.050385,0.121710,0.151800,0.106934,0.160935,0.132282,0.107986,0.136429,0.147373,0.138050,0.118910,0.075835,0.115064,0.145103,0.119851,0.089624,0.130583,0.062593,0.159568,0.099248,0.122628,0.101506,0.084387,0.064400,0.103713,0.164963,0.111077,0.148865,0.145103,0.092128,0.122628,0.130583,0.129725,0.153245,0.117003,0.130583,0.107986,0.056822,0.150340,0.105872,0.117003,0.113089,0.106934,0.081639,0.114081,0.124443,6.643171,3.973435
min,0.000000,0.000000,290.000000,6.350000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.185300,-122.505000,620.000000,651.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

## Load Model and Predict Price

In [6]:
with open("models/regression_model_rfe.pickle", "rb") as model:
    lr_model_rfe = pickle.load(model)

with open("models/transform_rfe.pickle", "rb") as transform:
    rfe_transform = pickle.load(transform)

In [7]:
# transform features according to RFECV
rfe_features = rfe_transform.transform(kc_test_df_features)

In [8]:
kc_price_predict_rfe = lr_model_rfe.predict(rfe_features)

In [9]:
price_prediction_rfe = pd.DataFrame({"price":kc_price_predict_rfe})

In [10]:
price_prediction_rfe.describe()

,price
count,4.323000e+03
mean,6.175983e+05
std,3.538496e+05
min,-4.107091e+05
25%,3.994175e+05
50%,5.552107e+05
75%,7.379058e+05
max,2.608024e+06


## Prediction Result Merge and Export

In [ ]:
selectedfeatures = []

In [ ]:
final_model.predict(holdout[sele])

In [11]:
kc_import_df = kc_import_df.merge(price_prediction_rfe, left_index=True, right_index=True)

In [12]:
#reset columns for export
kc_import_df = kc_import_df[['id', 'price', 'date', 'bedrooms', 'bathrooms', 'sqft_living',
                        'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
                        'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated','zipcode',
                        'lat', 'long','sqft_living15', 'sqft_lot15']]

In [13]:
kc_import_df.to_csv("results/kc_house_price_prediction.csv")

In [14]:
price_prediction_rfe.to_csv("results/kc_house_price_prediction_no_features.csv")